### Задача A. Что случилось?

Необходимо восстановить коэффициенты функции f(x), зная её значения на некотором наборе точек. При этом известно, что:

$$f(x) = ((a + dx) * e^x + (b + dx) * (x^(3/4)))^2 + (c + dx) * cos^2(x)$$

In [145]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit

In [146]:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

In [147]:
data = "C:\\Users\\HOME\\Documents\\Yandex\\case_A\\data.txt"

df = pd.read_csv(data, sep=' ', names=['X', 'y'])
df.head()

,X,y
0,3.692393,18960.598043
1,1.848369,2051.642981
2,1.479064,1314.926619
3,2.449248,4062.394248
4,2.473555,4175.464210


In [148]:
df.dtypes

X    float64
y    float64
dtype: object

### Способ 1 (аналитический через производную и OLS)

В исходных данных у нас есть признаки (1-ая колонка) и значение функции (2-ая колонка) в данной точке. Нужно найти коэффициенты функции аналитически.

In [149]:
X = df['X']
y = df['y']

In [150]:
len(X)

100000

In [151]:
X.head()

0    3.692393
1    1.848369
2    1.479064
3    2.449248
4    2.473555
Name: X, dtype: float64

In [152]:
y.shape

(100000,)

In [153]:
y.head()

0    18960.598043
1     2051.642981
2     1314.926619
3     4062.394248
4     4175.464210
Name: y, dtype: float64

Сначала определим функцию.

In [154]:
def f(x, a, b, c):
    da, db, dc = np.random.uniform(low=-0.001, high=0.001, size=3)
    return ((a + da) * np.exp(x) + (b + db) * x**(3/4))**2 + (c + dc) * np.cos(x)**2

In [155]:
y1 = f(2.0, 1.0, 2.0, 3.0)
y1

116.05651813123154

Вычислим производную функции $f(x)$:

In [156]:
# Определение производных функции f(x) по a, b и c
def df_da(x, a, b, da, db):
    return 2 * ((a + da) * np.exp(x) + (b + db) * x**(3/4)) * np.exp(x)

def df_db(x, a, b, da, db):
    return 3/2 * np.sqrt((a + da) * np.exp(x) + (b + db) * x**(3/4)) * (b + db) * x**(-1/4)

def df_dc(x, c, dc):
    return 2 * (c + dc) * np.cos(x) * (-np.sin(x)**2)

# Определение функции наименьших квадратов
def least_squares(X, y):
    # Инициализация начальных значений a, b, c
    a = np.random.uniform(low=-1, high=1)
    b = np.random.uniform(low=-1, high=1)
    c = np.random.uniform(low=-1, high=1)

    alpha = 0.01 # скорость обучения

    iterations = 10000 # количество итераций

    for i in range(iterations):
        # Вычисление градиентов производных
        grad_a = np.mean([2 * (f(x, a, b, c) - y[i]) * df_da(x, a, b, np.random.uniform(low=-0.001, high=0.001), np.random.uniform(low=-0.001, high=0.001))
                          for i, x in enumerate(X)])
        grad_b = np.mean([2 * (f(x, a, b, c) - y[i]) * df_db(x, a, b, np.random.uniform(low=-0.001, high=0.001), np.random.uniform(low=-0.001, high=0.001))
                          for i, x in enumerate(X)])
        grad_c = np.mean([2 * (f(x, a, b, c) - y[i]) * df_dc(x, c, np.random.uniform(low=-0.001, high=0.001))
                          for i, x in enumerate(X)])

        # Обновление значений a, b, c
        a = a - alpha * grad_a
        b = b - alpha * grad_b
        c = c - alpha * grad_c

    return a, b, c

In [135]:
# Вычисление коэффициентов a, b, c
a, b, c = least_squares(X, y)

# Вывод результатов
print(f'{a:.4f} {b:.4f} {c:.4f}')

KeyboardInterrupt: 

### Способ 2 (с библиотекой)

In [157]:
%%time
# Вычисление оптимальных значений весов a, b и c
popt, pcov = curve_fit(f, X, y)

CPU times: total: 125 ms
Wall time: 166 ms


In [158]:
# Вывод оптимальных значений весов a, b и c
a, b, c = popt
print('{:.6f} {:.6f} {:.6f}'.format(a, b, c))

1.060221 1.037162 0.960951


### Градиентный спуск

In [85]:
# Define the gradient of f(x) with respect to a, b, and c
def grad_f(x, a, b, c):
    da, db, dc = np.random.uniform(low=-0.001, high=0.001, size=3)
    df_da = 2 * ((a + da) * np.exp(x) + (b + db) * x**(3/4)) * np.exp(x)
    df_db = 3/2 * np.sqrt((a + da) * np.exp(x) + (b + db) * x**(3/4)) * (b + db) * x**(-1/4)
    df_dc = 2 * (c + dc) * np.cos(x) * (-np.sin(x)**2)
    return np.array([df_da, df_db, df_dc])

In [86]:
# Define the gradient descent function to find the optimal values of a, b, and c
def gradient_descent(X, y, learning_rate=0.01, epsilon=0.001, num_iterations=30000):
    # Initialize the values of a, b, and c
    a = np.random.uniform(low=-1, high=1)
    b = np.random.uniform(low=-1, high=1)
    c = np.random.uniform(low=-1, high=1)
    # Perform gradient descent
    for i in range(num_iterations):
        grad = np.zeros(3)
        for j in range(len(X)):
            grad += grad_f(X[j], a, b, c) * (f(X[j], a, b, c) - y[j])
        grad /= len(X)
        a -= learning_rate * grad[0]
        b -= learning_rate * grad[1]
        c -= learning_rate * grad[2]
        if np.linalg.norm(grad) < epsilon:
            break
    return a, b, c

In [87]:
# Find the optimal values of a, b, and c using gradient descent
a, b, c = gradient_descent(X, y)

# Print the optimal values of a, b, and c
print('{:.6f} {:.6f} {:.6f}'.format(a, b, c))

C:\Users\HOME\AppData\Local\Temp\ipykernel_6732\3886243238.py:5: RuntimeWarning: invalid value encountered in sqrt
  df_db = 3/2 * np.sqrt((a + da) * np.exp(x) + (b + db) * x**(3/4)) * (b + db) * x**(-1/4)


KeyboardInterrupt: 

In [122]:
%%time
# Получение первых 10000 строк данных
X = X[:10000]
y = y[:10000]

# Convert X Series to numpy array
X_arr = np.array(X)

# Reshape X_arr
X_arr = X_arr.reshape(-1, 1)

scaler = StandardScaler()

# Convert X Series to numpy array for normalization
X_norm = np.array(X)
X_norm = scaler.fit_transform(X_norm.reshape(-1, 1))

# Нормализация данных
X_norm = scaler.transform(X_norm.reshape(-1, 1))

# Настройка параметров для градиентного спуска
learning_rate = 0.01
max_iterations = 10000
epsilon = 0.001

# Преобразование max_iterations в целое число
import math
max_iterations = math.ceil(max_iterations)

# Вычисление оптимальных значений весов a, b и c
a, b, c = gradient_descent(X, y, learning_rate, epsilon, num_iterations=max_iterations)

# Вывод оптимальных значений весов a, b и c
print('{:.6f} {:.6f} {:.6f}'.format(a, b, c))

C:\Users\HOME\AppData\Local\Temp\ipykernel_6732\3886243238.py:5: RuntimeWarning: invalid value encountered in sqrt
  df_db = 3/2 * np.sqrt((a + da) * np.exp(x) + (b + db) * x**(3/4)) * (b + db) * x**(-1/4)


KeyboardInterrupt: 